In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch.unitroot import ADF, PhillipsPerron
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf

import warnings
warnings.filterwarnings('ignore')

In [2]:
household_power_consumption = pd.read_csv("household_power_consumption.txt", sep=';',
                                                parse_dates={'DateTime' : ['Date', 'Time']}, 
                                                index_col=["DateTime"],
                                                dayfirst=True,
                                                low_memory=False,
                                                na_values=['nan','?'])


FileNotFoundError: [Errno 2] No such file or directory: 'household_power_consumption.txt'

In [ ]:
household_power_consumption = household_power_consumption[household_power_consumption.index.year > 2006]
household_power_consumption = household_power_consumption[household_power_consumption.index.year < 2010]
household_power_consumption = household_power_consumption.fillna(method='ffill')

## Creating daily weekly monthly and quarterly Dataframes


In [ ]:
household_power_consumption = household_power_consumption["Global_active_power"]
household_power_consumption = household_power_consumption.dropna()
daily = household_power_consumption.resample("D").mean()
# daily.to_csv("daily.csv")
weekly = household_power_consumption.resample("W").mean()
# weekly.to_csv("weekly.csv")
monthly = household_power_consumption.resample("M").mean()
# monthly.to_csv("monthly.csv")
quarterly = household_power_consumption.resample("3M").mean()
# quarterly.to_csv("quarterly.csv")

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 6))
daily.plot(y="Global_active_power", ax=axes[0,0], title="Daily")
weekly.plot(y="Global_active_power", ax=axes[0,1], title="Weekly")
monthly.plot(y="Global_active_power", ax=axes[1,0], title="Monthly")
quarterly.plot(y="Global_active_power", ax=axes[1,1], title="Quarterly")
plt.tight_layout()
plt.plot()

## Decompose Trend and Seasonality  

In [ ]:
def decompose(time_period, period):
    decomposition = sm.tsa.seasonal_decompose(time_period, model='additive', period=period)
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 6))
    axes[0].plot(decomposition.trend)
    axes[0].set_title("Trend")
    axes[1].plot(decomposition.seasonal)
    axes[1].set_title("Seasonality")
    axes[2].plot(decomposition.resid)
    axes[2].set_title("Residual")
    plt.tight_layout()
    plt.plot()
    print(ADF(decomposition.resid.dropna()))
    #print(PhillipsPerron(decomposition.resid.dropna()))
    return decomposition.resid.dropna()

In [ ]:
daily_resid = decompose(daily, 365)

In [ ]:
weekly_resid = decompose(weekly, 53)

In [ ]:
monthly_resid = decompose(monthly, 12)

In [ ]:
quarterly_resid = decompose(quarterly, 4)

In [ ]:
def test_arima(timeseries):
    order_values = [(p, d, q) for p in range(4) for d in range(2) for q in range(4)]
    best_aicc = float('inf')
    best_order = None

    for p in range(4):
        for d in range(2):
            for q in range(4):
                model = ARIMA(timeseries, order=(p, d, q))
                results = model.fit()
                aicc = results.aicc
                if aicc < best_aicc:
                    best_aicc = aicc
                    best_order = (p, d, q)
        
    return best_order, best_aicc


In [ ]:
print("Best daily ARIMA parameters & AICC:")
test_arima(daily_resid)

In [ ]:
print("Best weekly ARIMA parameters & AICC:")
test_arima(weekly_resid)

In [ ]:
print("Best monthly ARIMA parameters & AICC:")
test_arima(monthly_resid)

In [ ]:
print("Best quarterly ARIMA parameters & AICC:")
test_arima(quarterly_resid)

# Quality Model Check 

In [ ]:
from scipy.stats import shapiro
model = ARIMA(daily_resid, order=(3, 0, 2)).fit()
model.plot_diagnostics()
print(model.params)
print(model.summary())
print(shapiro(model.standardized_forecasts_error))


In [ ]:
model2 = ARIMA(weekly_resid, order=(2, 0, 2)).fit()
model2.plot_diagnostics()
print(model2.params)
print(model2.summary())
print(shapiro(model2.standardized_forecasts_error))

In [ ]:
model3 = ARIMA(monthly_resid, order=(0, 0, 1)).fit()
model3.plot_diagnostics()
print(model3.params)
print(model3.summary())
print(shapiro(model3.standardized_forecasts_error))

In [ ]:
model4 = ARIMA(quarterly_resid, order=(0, 0, 1)).fit()
#model4.plot_diagnostics()
print(model4.params)
print(model4.summary())

print(shapiro(model4.standardized_forecasts_error))

## Forecast

In [ ]:
forecast = model.get_forecast(steps=28) 
forecast_values = forecast.predicted_mean
confidence_interval = forecast.conf_int()  
plt.figure(figsize=(12, 6))
plt.plot(daily_resid, label='Series')
plt.plot(forecast_values.index, forecast_values, color='red', label='Forecast')
plt.fill_between(confidence_interval.index, confidence_interval.iloc[:, 0], confidence_interval.iloc[:, 1], color='gray', alpha=0.3, label='Conf Interval')
plt.legend()
plt.show()

In [ ]:
forecast = model2.get_forecast(steps=25) 
forecast_values = forecast.predicted_mean
confidence_interval = forecast.conf_int()  
plt.figure(figsize=(12, 6))
plt.plot(weekly_resid, label='Series')
plt.plot(forecast_values.index, forecast_values, color='red', label='Forecast')
plt.fill_between(confidence_interval.index, confidence_interval.iloc[:, 0], confidence_interval.iloc[:, 1], color='gray', alpha=0.3, label='Conf Interval')
plt.legend()
plt.show()

In [ ]:
forecast = model3.get_forecast(steps=10) 
forecast_values = forecast.predicted_mean
confidence_interval = forecast.conf_int()  
plt.figure(figsize=(12, 6))
plt.plot(monthly_resid, label='Series')
plt.plot(forecast_values.index, forecast_values, color='red', label='Forecast')
plt.fill_between(confidence_interval.index, confidence_interval.iloc[:, 0], confidence_interval.iloc[:, 1], color='gray', alpha=0.3, label='Conf Interval')
plt.legend()
plt.show()

In [ ]:
forecast = model4.get_forecast(steps=3) 
forecast_values = forecast.predicted_mean
confidence_interval = forecast.conf_int()  
plt.figure(figsize=(12, 6))
plt.plot(quarterly_resid, label='Series')
plt.plot(forecast_values.index, forecast_values, color='red', label='Forecast')
plt.fill_between(confidence_interval.index, confidence_interval.iloc[:, 0], confidence_interval.iloc[:, 1], color='gray', alpha=0.3, label='Conf Interval')
plt.legend()
plt.show()

# To check

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
plot_acf(daily_resid, ax=ax)
ax.set_xlabel('Lag')
ax.set_ylabel('Autocorrelation')
ax.set_title('Daily Autocorrelation Function')
plt.show()
daily_count=len(daily_resid)
print('the number of values is:',daily_count)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
plot_acf(weekly_resid, ax=ax)
ax.set_xlabel('Lag')
ax.set_ylabel('Autocorrelation')
ax.set_title('Weekly Autocorrelation Function')
plt.show()

weekly_count=len(weekly_resid)
print('the number of values is:',weekly_count)



In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
fig, ax = plt.subplots(figsize=(10, 5))
plot_acf(monthly_resid, ax=ax)
ax.set_xlabel('Lag')
ax.set_ylabel('Autocorrelation')
ax.set_title('Monthly Autocorrelation Function')
plt.show()
monthly_count=len(monthly_resid)
print('the number of values is:',monthly_count)

In [ ]:

fig, ax = plt.subplots(figsize=(10, 5))
plot_acf(quarterly_resid, ax=ax)
ax.set_xlabel('Lag')
ax.set_ylabel('Autocorrelation')
ax.set_title('Quarterly Autocorrelation Function')
plt.show()
quarterly_count=len(quarterly_resid)
print('the number of values is:', quarterly_count)